# Notebook criado em 22/08/20 para desenvolvimento de uma metodologia para análise da correlação entre focos do satélite de referência (AQUA_M-T) como os S-NPP

In [76]:
# Importação de bibliotecas de referência
import pandas as pd
from sqlalchemy import create_engine, pool
from scipy.stats import linregress
import seaborn as sns
%matplotlib inline


In [ ]:
pd.options.display.float_format = '{:,.2f}'.format


In [77]:
# Conexão com o banco de dados
# engine = create_engine('postgresql://@localhost:5432/api', poolclass=pool.NullPool)
engine = create_engine('postgresql://@localhost:9933/api', poolclass=pool.NullPool)

<pre>
api=# select name_1, id_1 from dados_geo.estados where id_0 = 33;
       name_1        | id_1
---------------------+------
 MATO GROSSO         |   51
 GOIÁS               |   52
 DISTRITO FEDERAL    |   53
 RONDÔNIA            |   11
 ACRE                |   12
 AMAZONAS            |   13
 RORAIMA             |   14
 PARÁ                |   15
 AMAPÁ               |   16
 TOCANTINS           |   17
 MARANHÃO            |   21
 PIAUÍ               |   22
 CEARÁ               |   23
 PARAÍBA             |   25
 RIO GRANDE DO NORTE |   24
 PERNAMBUCO          |   26
 ALAGOAS             |   27
 SERGIPE             |   28
 BAHIA               |   29
 MINAS GERAIS        |   31
 ESPÍRITO SANTO      |   32
 RIO DE JANEIRO      |   33
 SÃO PAULO           |   35
 PARANÁ              |   41
 SANTA CATARINA      |   42
 RIO GRANDE DO SUL   |   43
 MATO GROSSO DO SUL  |   50
 
 
 api=# select distinct bioma from collection2.focos_bdq_c2 where id_0 =33 and data_hora_gmt > '20200801';
     bioma
----------------
 Amazônia
 Caatinga
 Cerrado
 Mata Atlântica
 Pampa
 Pantanal
 
 </pre>

In [ ]:
# CONSULTA PARA FAZER AS ANALISES PARA O BRASIL
engine.connect()
sql = """

select
	 date_trunc('month', data_hora_gmt)::date data_mes,
	satelite,
	count(1) as qtd
from
	collection2.focos_bdq_c2
where
	--extract(month from data_hora_gmt) = 1 and 
	data_hora_gmt >= '20130101' and data_hora_gmt < '20200816'
	and satelite in ('AQUA_M-T', 'NPP-375')
	and id_0 = 33
	--and id_1 = 52
	AND (id_tipo_area_industrial = 0 or id_tipo_area_industrial is null) 
    and id_area_industrial = 0
group by 1,2
order by 1,2

;
"""
focos = pd.read_sql(sql, engine)
focos["data_mes"] = pd.to_datetime(focos.data_mes)

In [ ]:
# CONSULTA PARA FAZER AS ANALISES PARA OS BIOMAS

engine.connect()
sql = """

select
	 date_trunc('month', data_hora_gmt)::date data_mes,
	satelite,
    bioma,
	count(1) as qtd
from
	collection2.focos_bdq_c2
where
	--extract(month from data_hora_gmt) = 1 and 
	data_hora_gmt >= '20130101' and data_hora_gmt < '20200816'
	and satelite in ('AQUA_M-T', 'NPP-375')
	and id_0 = 33
	--and id_1 = 52
	AND (id_tipo_area_industrial = 0 or id_tipo_area_industrial is null)
    and id_area_industrial = 0
group by 1,2,3
order by 1,2,3

;
"""
focos = pd.read_sql(sql, engine)
focos["data_mes"] = pd.to_datetime(focos.data_mes)

In [ ]:
# CONSULTA PARA FAZER AS ANALISES PARA OS ESTADOS

engine.connect()
sql = """

select
	 date_trunc('month', data_hora_gmt)::date data_mes,
	satelite,
    id_1,
	count(1) as qtd
from
	collection2.focos_bdq_c2
where
	data_hora_gmt >= '20130101' and data_hora_gmt < '20200816'
	and satelite in ('AQUA_M-T', 'NPP-375')
	and id_0 = 33
	AND (id_tipo_area_industrial = 0 or id_tipo_area_industrial is null)
    and id_area_industrial = 0
group by 1,2,3
order by 1,2,3

;
"""
focos = pd.read_sql(sql, engine)
focos["data_mes"] = pd.to_datetime(focos.data_mes)

In [80]:
# CONSULTA GERAL

engine.connect()
sql = """

select
    date_trunc('month', data_hora_gmt)::date data_mes,
    satelite,
    id_1,
    estado,
    bioma,
    count(1) as qtd
from
	collection2.focos_bdq_c2
where
	data_hora_gmt >= '20120101' and data_hora_gmt < '20200816'
	and satelite in ('AQUA_M-T', 'NPP-375', 'NOAA-12')
	and id_0 = 33
	AND (id_tipo_area_industrial = 0 or id_tipo_area_industrial is null)
    and id_area_industrial = 0
group by 1,2,3,4,5
order by 1,2

;
"""
focos = pd.read_sql(sql, engine)
focos["data_mes"] = pd.to_datetime(focos.data_mes)

In [ ]:
focos.info()

In [81]:
focos.to_csv('./dados_geral.csv')

In [ ]:
focos = pd.read_csv('./dados_biomas.csv', index_col=0, parse_dates=[1])

In [ ]:
focos.head(-1)

In [ ]:
focos = focos[focos.bioma == 'Amazônia'][["data_mes", "satelite", "qtd"]]

In [ ]:
acum = focos.pivot_table(index='data_mes', columns=[ 'satelite'], aggfunc="mean", values='qtd', margins=False )
acum = pd.DataFrame(acum.to_records())

In [ ]:
acum.head()

In [ ]:
acum.info()

In [ ]:
acum['ano']=acum.data_mes.dt.year
acum.head()

In [ ]:
# sns.pairplot(data, x_vars=['TV','Radio','Newspaper'], y_vars='Sales')
#sns.pairplot(acum,x_vars=['NPP-375'], y_vars=['AQUA_M-T'],height=8.27, aspect=11.7/8.27);
# sns.lmplot(x="total_bill", y="tip", col="day", data=tips, col_wrap=2, height=3);
sns.lmplot(x="NPP-375", y="AQUA_M-T", col="ano", data=acum, col_wrap=4 , height=3);

In [ ]:
g = sns.jointplot('NPP-375', 'AQUA_M-T', data=acum, kind="reg",
                  color="r", height=8.27)

In [ ]:
linregress(acum['NPP-375'] , acum['AQUA_M-T'])

In [ ]:
slope, intercept, r_value, p_value, std_err = linregress( acum['NPP-375'], acum['AQUA_M-T'] )

In [ ]:
print("slope: %f    intercept: %f" % (slope, intercept))

In [ ]:
print("R-squared: %f" % r_value**2)

In [ ]:
acum['ref_calc'] = acum['NPP-375']*slope + intercept

In [ ]:
int(acum['AQUA_M-T'].sum()), int(acum['ref_calc'].sum())

 O valor acima mostra que a soma de todo o período deu uma correlação perfeita. Porém na celula abaixo nota-se que os valores acumulados mensais estimados e observados em 2020, mes a mes são muito diferente

In [ ]:
acum [(acum.data_mes.dt.year == 2020)]

In [ ]:
total20_aqua = acum [(acum.data_mes.dt.year == 2020)]["AQUA_M-T"].sum()
total20_calc = int(acum [(acum.data_mes.dt.year == 2020)]["ref_calc"].sum())
dif = (total20_calc - total20_aqua) / total20_aqua 
print(total20_aqua, total20_calc, dif)

 Optei por fazer uma análise considerando a quantidade de focos.
 
 Neste sentido fiz uma análise estatística da distribuição do total de focos nos meses.

In [ ]:
# focos.acum_ano.hist(figsize=(20,10), bins=np.arange(0,40000,25));
focos.qtd.hist();

In [ ]:
focos.qtd.describe(percentiles=[0.10,0.25,0.50,0.75,0.90])

In [ ]:
focos.boxplot(column='qtd',
              figsize=(5,10)
);

  Pelo fato da estatística te apresentado que 90% dos meses possuem quantidade menor que 15 mil focos, optei por fazer um recorte e analisar este sub conjunto primeiro.

In [ ]:
f15 = focos[focos.qtd <= 15000]

In [ ]:
f15

In [ ]:
f15.boxplot(column='qtd',
              figsize=(5,10)
);

In [ ]:
acum = f15.pivot_table(index='data_mes', columns=['satelite'], aggfunc="mean", values='qtd', margins=False )
acum = pd.DataFrame(acum.to_records())
acum

  Para ter certeza que ambos os satélites possuem registros em todos os meses fiz o teste abaixo, e verifiquei que o NPP possui 21 meses com mais de 15 focos enquanto o aqua possui menos.

In [ ]:
acum.isnull().sum()

In [ ]:
acum[acum["NPP-375"].isnull()]

In [ ]:
acum = acum[~(acum["NPP-375"].isnull() )]

In [ ]:
acum = acum[~(acum.data_mes.dt.year == 2020)]

In [ ]:
g = sns.jointplot('NPP-375', 'AQUA_M-T', data=acum, kind="reg",
                  color="r", height=8.27)

In [ ]:
linregress(acum['NPP-375'] , acum['AQUA_M-T'])

In [ ]:
0.9328491117829526**2

  Como pode ser notado o R2 desta analise não foi satisfatório e portanto vou tentar a abordagem de fazer por trimeste.
  
# Analise por trimestre

In [ ]:
focos

In [ ]:
focos_mes = focos[focos.data_mes.dt.month.isin([5,6,7,11,12])]

In [ ]:
focos_mes = focos[focos.data_mes.dt.month.isin([1,2,3])]

In [ ]:
focos_mes = focos[focos.data_mes.dt.month.isin([8,9,10])]

In [ ]:
focos_mes = focos[focos.data_mes.dt.month.isin([5,6,7,11,12,8,9,10])] 

In [ ]:
focos20 = focos[(focos.data_mes.dt.year == 2020)]
acum20 = focos20.pivot_table(index='data_mes', columns=['satelite'], aggfunc="mean", values='qtd', margins=False )
acum20 = pd.DataFrame(acum20.to_records())
acum20

In [ ]:
acum = focos_mes.pivot_table(index='data_mes', columns=['satelite'], aggfunc="mean", values='qtd', margins=False )
acum = pd.DataFrame(acum.to_records())

acum = acum[~(acum.data_mes.dt.year == 2020)]

acum.describe()

In [ ]:
acum['mes']=acum.data_mes.dt.month

In [ ]:
acum

In [ ]:
acum.plot.scatter('NPP-375', 'AQUA_M-T', c='mes',colormap='viridis');
# g = sns.jointplot('NPP-375', 'AQUA_M-T', data=acum, kind="reg", color="r", height=8.27)

In [ ]:
linregress(acum['NPP-375'] , acum['AQUA_M-T'])

In [ ]:
a.rvalue**2

In [ ]:
a = linregress(acum['NPP-375'] , acum['AQUA_M-T'])

In [ ]:
acum20['ref_calc'] = acum20['NPP-375']*a.slope + a.intercept

In [ ]:
acum20

In [ ]:
acum20['AQUA_M-T'].sum(),int(acum20['ref_calc'].sum())

In [ ]:
total20_aqua = acum20 [(acum20.data_mes.dt.year == 2020)]["AQUA_M-T"].sum()
total20_calc = int(acum20 [(acum20.data_mes.dt.year == 2020)]["ref_calc"].sum())
dif = (total20_calc - total20_aqua) / total20_aqua 
print('mes1_5' , total20_aqua, total20_calc, dif)

## Conclusão  
  Conforme foi testado todos os valores considerando os meses mais criticos e mesnos criticos, com melhor r2 todos não conseguem fazer uma estimativa mensal que seja interessante.
  
  Existem ainda duas abordagem uma de separar apenas o ano anterior como referencia e outra considenrando as variações de dia.
  
  # Análise considerando apenas o ano anterior

In [ ]:
focos

In [ ]:
focos19 = focos[focos.data_mes.dt.year == 2019]

In [ ]:
acum = focos19.pivot_table(index='data_mes', columns=['satelite'], aggfunc="mean", values='qtd', margins=False )
acum = pd.DataFrame(acum.to_records())

acum.describe()

In [ ]:
g = sns.jointplot('NPP-375', 'AQUA_M-T', data=acum, kind="reg",
                  color="r", height=8.27)

In [ ]:
acum

In [ ]:
linregress(acum['NPP-375'] , acum['AQUA_M-T'])

In [ ]:
a = linregress(acum['NPP-375'] , acum['AQUA_M-T'])

In [ ]:
a.rvalue**2

In [ ]:
a = linregress(acum['NPP-375'] , acum['AQUA_M-T'])

In [ ]:
focos20 = focos[(focos.data_mes.dt.year == 2020)]
acum20 = focos20.pivot_table(index='data_mes', columns=['satelite'], aggfunc="mean", values='qtd', margins=False )
acum20 = pd.DataFrame(acum20.to_records())
acum20

In [ ]:
acum20['ref_calc'] = acum20['NPP-375']*a.slope + a.intercept

In [ ]:
acum20

In [ ]:
total20_aqua = acum20 [(acum20.data_mes.dt.year == 2020)]["AQUA_M-T"].sum()
total20_calc = int(acum20 [(acum20.data_mes.dt.year == 2020)]["ref_calc"].sum())
dif = (total20_calc - total20_aqua) / total20_aqua 
print('mes1_5' , total20_aqua, total20_calc, dif)

## Conclusão  
  Conforme foi testado utilizando como referencia os valores do ano anterior foi um bom melhor r2 e a diferença do acumulado anual foi de -18%
  
  # Análise considerando parametros do Alberto

In [ ]:
focos

In [ ]:
focos20 = focos[(focos.data_mes >= '2019-08-01')]
acum = focos20.pivot_table(index='data_mes', columns=['satelite'], aggfunc="mean", values='qtd', margins=False )
acum = pd.DataFrame(acum.to_records())
acum

In [ ]:
acum.isnull().sum()

In [ ]:
acum.describe()

In [ ]:
g = sns.jointplot('NPP-375', 'AQUA_M-T', data=acum, kind="reg", xlim=(0,450000), ylim=(0,60000),
                  color="r", height=8.27)

In [ ]:
ax = sns.regplot(x='NPP-375', y='AQUA_M-T', data=acum, marker="+", )

In [ ]:
import statsmodels.api as sm

In [ ]:
x = sm.add_constant(acum['NPP-375'])
y = acum['AQUA_M-T']

In [ ]:
results = sm.OLS(y,x).fit()

In [ ]:
results.summary()

In [ ]:
linregress(acum['NPP-375'] , acum['AQUA_M-T'])

In [ ]:
a = linregress(acum['NPP-375'] , acum['AQUA_M-T'])

In [ ]:
a.rvalue**2

In [ ]:
focos20 = focos[(focos.data_mes.dt.year == 2020)]
acum20 = focos20.pivot_table(index='data_mes', columns=['satelite'], aggfunc="mean", values='qtd', margins=False )
acum20 = pd.DataFrame(acum20.to_records())
acum20

In [ ]:
acum20['ref_calc'] = acum20['NPP-375']*a.slope + a.intercept

In [ ]:
acum20

In [ ]:
total20_aqua = acum20 [(acum20.data_mes.dt.year == 2020)]["AQUA_M-T"].sum()
total20_calc = int(acum20 [(acum20.data_mes.dt.year == 2020)]["ref_calc"].sum())
dif = (total20_calc - total20_aqua) / total20_aqua 
print('mes1_5' , total20_aqua, total20_calc, dif)